In [1]:
import time
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sentence_transformers.util import cos_sim
import sys
import key_words

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
from typing import Tuple

In [8]:
cols_names_t:dict={"students_emb": Tuple[str or None, bool],
                    "students_answers":Tuple[str or None, bool],
                    "model_answers_emb":Tuple[str or None, bool],
                    "model_answers":Tuple[str or None, bool],
                    "siamese_sores":Tuple[str or None, bool],
                    "students_keywords":Tuple[str or None, bool],
                    "students_keywords_emb":Tuple[str or None, bool],
                    "model_keywords":Tuple[str or None, bool],
                    "model_keywords_emb":Tuple[str or None, bool],
                    "keywords_scores":Tuple[str or None, bool],
                    "ner_keywords":Tuple[str or None, bool],
                    "ner_keywords_scores":Tuple[str or None, bool],
                    "labels":Tuple[str or None, bool],}

In [9]:
cols_names:cols_names_t = {"students_emb": ("students_emb", True),
                    "students_answers":("EssayText", True),
                    "model_answers_emb":("model_answers_emb", True),
                    "model_answers":("model_answers", True),
                    "siamese_sores":(None, True),
                    "students_keywords":("students_keywords", True),
                    "students_keywords_emb":("students_keywords_emb", True),
                    "model_keywords":("model_keywords", True),
                    "model_keywords_emb":("model_keywords_emb", True),
                    "keywords_scores":("keywords_scores", True),
                    "ner_keywords":("ner_keywords", True),
                    "ner_keywords_scores":("ner_keywords_scores", True),
                    "labels":("labels", True),}

In [2]:
train_path = "data/train_phase1.tsv"
df = pd.read_csv(train_path, sep="\t")

In [3]:
import pickle
def save_obj(obj:object,name:str):
    ext = '.pickle'
    with open(name + ext, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(name:str)->object:
    ext = '.pickle'
    with open(name + ext, 'rb') as handle:
        return pickle.load(handle)

In [4]:
import transformer_model
import grading_model

BERT = transformer_model.BERTModel()

GM = grading_model.GradingModel(BERT)

In [ ]:
class FineTuining(grading_model.GradingModel):
    def __init__(self, BERT):
        super().__init__(BERT)
        self.BERT = BERT
        self.model = self.BERT.model
        self.tokenizer = self.BERT.tokenizer
        self.device = self.BERT.device
        self.model.to(self.device)
        self.model.eval()
        self.tokenizer.to(self.device)

    # def predict(self, text):
    #     tokens = self.tokenizer.encode(text)
    #     tokens = tokens.unsqueeze(0)
    #     with torch.no_grad():
    #         tokens = tokens.to(self.device)
    #         outputs = self.model(tokens)
    #         outputs = torch.nn.functional.softmax(outputs, dim=2)
    #         outputs = outputs.squeeze(0)
    #         outputs = outputs.cpu().numpy()
    #         outputs = np.argmax(outputs, axis=1)
    #         outputs = self.tokenizer.decode(outputs)
    #     return outputs
    def pre_fit(self, df,students_emb,model_answers_emb,
            top_n=6,diversity=0.5,n_gram_range=(1,2),
            threshold=0.5,exception_entites=None,
                cols_names:dict={
                    "students_emb": Tuple[str or None, bool],
                    "students_answers":Tuple[str or None, bool],
                    "model_answers_emb":Tuple[str or None, bool],
                    "model_answers":Tuple[str or None, bool],
                    "siamese_scores":Tuple[str or None, bool],
                    "students_keywords":Tuple[str or None, bool],
                    "students_keywords_emb":Tuple[str or None, bool],
                    "model_keywords":Tuple[str or None, bool],
                    "model_keywords_emb":Tuple[str or None, bool],
                    "keywords_scores":Tuple[str or None, bool],
                    "ner_keywords":Tuple[str or None, bool],
                    "ner_keywords_scores":Tuple[str or None, bool],
                    "labels":Tuple[str or None, bool],}):
        
        """
        pre-fit the model with dataframe

        Args:
            df (pd.DataFrame): dataframe
            cols_names (dict): dictionary of columns names
                key is column name
                value is tuple of (string, bool)
                    (column name or None, is_column_name_needed)

        """
        for col,col_name,is_needed in cols_names.items():
            if is_needed:
                if col_name is None:
                    col_name = col
                df[col_name] = df[col]

        self

        return self

In [4]:
s = "haszem a kutya"
util = cos_sim(BERT.model.encode(s), np.eye(768,dtype=np.float32))
util.shape
np.eye(768,dtype=np.float32).shape
eye = np.eye(768,dtype=np.float32)
emb = BERT.model.encode(s).reshape(1, -1)
sk = cosine_similarity(emb,eye)
n = cosine_similarity(emb)
cos = cos_sim(emb, emb)
cos
# get only 5 digits after decimal point
np.array(cos).clip(-1, 1).round(6)
n
s = ".4"
s2 = "4"
cos_sim(BERT.model.encode(s), BERT.model.encode(s2))
cosine_similarity(BERT.model.encode(s).reshape(1, -1), BERT.model.encode(s2).reshape(1, -1))

## optimize match_keywords function last output grade

In [135]:
%timeit reduce(lambda x,y: x+y,map(fn_, similarities))

5.48 ms ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [131]:

%timeit sum(map(fn_, similarities))

5.45 ms ± 52.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [129]:
%timeit np.sum(np.array(list(map(fn_, similarities))))

5.51 ms ± 59.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# optimize all keywords file

In [70]:
df_emb = load_obj("data/ess_emb")

In [5]:
ess_1_model_answers = load_obj("data/essaySet_1_model_answers")
ess_2_model_answers = load_obj("data/essaySet_2_model_answers")
ess_3_model_answers = load_obj("data/essaySet_3_model_answers")
ess_4_model_answers = load_obj("data/essaySet_4_model_answers")
ess_5_model_answers = load_obj("data/essaySet_5_model_answers")
ess_6_model_answers = load_obj("data/essaySet_6_model_answers")
ess_7_model_answers = load_obj("data/essaySet_7_model_answers")
ess_8_model_answers = load_obj("data/essaySet_8_model_answers")
ess_9_model_answers = load_obj("data/essaySet_9_model_answers")
ess_10_model_answers = load_obj("data/essaySet_10_model_answers")

In [9]:
ess_1_model_answers_emb = BERT.model.encode(ess_1_model_answers)
ess_2_model_answers_emb = BERT.model.encode(ess_2_model_answers)
ess_3_model_answers_emb = BERT.model.encode(ess_3_model_answers)
ess_4_model_answers_emb = BERT.model.encode(ess_4_model_answers)
ess_5_model_answers_emb = BERT.model.encode(ess_5_model_answers)
ess_6_model_answers_emb = BERT.model.encode(ess_6_model_answers)
ess_7_model_answers_emb = BERT.model.encode(ess_7_model_answers)
ess_8_model_answers_emb = BERT.model.encode(ess_8_model_answers)
ess_9_model_answers_emb = BERT.model.encode(ess_9_model_answers)
ess_10_model_answers_emb = BERT.model.encode(ess_10_model_answers)

In [ ]:
emb_dict = {'ess_1_model_answers_emb': ess_1_model_answers_emb,
 'ess_2_model_answers_emb': ess_2_model_answers_emb,
 'ess_3_model_answers_emb': ess_3_model_answers_emb,
 'ess_4_model_answers_emb': ess_4_model_answers_emb,
 'ess_5_model_answers_emb': ess_5_model_answers_emb,
 'ess_6_model_answers_emb': ess_6_model_answers_emb,
 'ess_7_model_answers_emb': ess_7_model_answers_emb,
 'ess_8_model_answers_emb': ess_8_model_answers_emb,
 'ess_9_model_answers_emb': ess_9_model_answers_emb,
 'ess_10_model_answers_emb': ess_10_model_answers_emb}
save_obj(emb_dict, "data/model_answer_emb")

In [6]:
emb_dict = load_obj("data/model_answer_emb")

In [14]:
docs = df.query('EssaySet == 1')["EssayText"].values.tolist()
model_candidates = list(map(lambda ans: key_words.candidates_tokens(ans,n_gram_range=(1,2)),ess_1_model_answers))
# model_candidate_emb = BERT.model.encode(model_candidates)
# emb_dict[ess_1_model_answers_emb]
model_candidate_emb = list(map(lambda  cand:BERT.model.encode(cand),model_candidates))


l = zip(emb_dict['ess_1_model_answers_emb'],model_candidate_emb,model_candidates)

In [16]:
save_obj(l, "data/ess_1_model_answers_emb__candidates")

In [7]:
def maximal_marginal_relevance(doc_embedding: np.ndarray,
        word_embeddings: np.ndarray,
        words,
        top_n = 5,
        diversity = 0.8):
    """
    Maximal Marginal Relevance algorithm for keyword extraction
    * from KeyBERT repository on github

    Args:
        doc_embedding (numpy.ndarray): embedding of shape (1, 768)
        word_embeddings (numpy.ndarray): embedding of shape (N, 768)
        words (List[str]): list of words
        top_n (Optional[int]): number of top words to extract
        diversity (Optional[float]): diversity of top words to extract

    Returns:
        List[Tuple[str, float]]: list of top_n words with their scores
    """
    # make sure 2d array
    if doc_embedding.ndim == 1:
        doc_embedding = doc_embedding.reshape(1, -1)

    # Extract similarity within words, and between words and the document

    word_doc_similarity = np.array(cos_sim(word_embeddings, doc_embedding)).clip(-1, 1).round(6)
    word_similarity = np.array(cos_sim(word_embeddings, word_embeddings)).clip(-1, 1).round(6)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate maximal_marginal_relevance
        mmr = (1-diversity) * candidate_similarities -\
            diversity * target_similarities.reshape(-1, 1)
        # if return mmr is empty
        if mmr.size == 0:
            continue
        mmr = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr)
        candidates_idx.remove(mmr)

    return [words[idx] for idx in keywords_idx]

In [8]:
def emb_keywords(keywords):
    x = np.array(list(map(lambda k:
     np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords)))
    if x.ndim == 3:
        x = x.reshape(max(x.shape[1],x.shape[0]),x.shape[2])
    # print("emb",x.shape)
    return x
    # return list(map(lambda k: np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords))

In [18]:
keywords = list(map(lambda x: maximal_marginal_relevance(x[0].reshape(1, -1),x[1],x[2]),zip(emb_dict['ess_1_model_answers_emb'],model_candidate_emb,model_candidates)))
print(len(keywords))
print(keywords)

22
[['know vinegar', 'container', 'need', 'vinegar used', 'used'], ['type vinegar', 'need know', 'container', 'used', 'vinegar used'], ['materials test', 'need know', 'know materials', 'test', 'know'], ['materials used', 'need know', 'size', 'surface area', 'know'], ['sample rinsed', 'need know', 'distilled water', 'long', 'sample'], ['know drying', 'method use', 'need know', 'drying method', 'know'], ['type container', 'need know', 'know size', 'use', 'container use'], ['replicate experiment', 'information need', 'vinegar poured', 'type containers', 'completely different'], ['know vinegar', 'additional information', 'putt sample', 'needed need', 'long'], ['samples vinegar', 'information need', 'replicate experiment', 'materials different', 'completely submerged'], ['students didn', 'vinegar add', 'referring secondly', 'specify samples', 'step need'], ['major experiment', 'know vinegar', 'containers use', 'poured lastly', 'material thats'], ['different containers', 'experiment need', '

In [22]:


# keywords_emb = model.encode(keywords)

students_candidates = list(map(lambda n_grams:list(map(lambda doc:key_words.get_candidates(n_grams, doc),docs)),students_n_grams))

# students_candidates is len 22         models
# students_candidates[0] is len 1672    essays
# students_candidates[0][0] is len 2    ngrams
# students_candidates[0][0][0] len 40   candidates


Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error:

In [127]:
def emb_keywords(keywords):
    # return np.array(list(map(lambda k: np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords)))
    return np.array(list(map(lambda k: np.array(list(map(lambda x: get_words_emb(str(x)), k))),keywords)))
    # return np.array(list(map(BERT.model.encode, keywords)))
def old_emb_keywords(keywords):
    return np.array(list(map(BERT.model.encode, keywords)))

In [ ]:
y = list(map(lambda st: list(map(lambda x: len(x), st)),students_candidates))

In [ ]:
len(y),len(y[0]),y[0][0]

(22, 1672, 2)

In [52]:
keywords = list(map(lambda x: maximal_marginal_relevance(x[0].reshape(1, -1),x[1],x[2]),zip(emb_dict['ess_1_model_answers_emb'][:2],model_candidate_emb[:2],model_candidates[:2])))
students_n_grams = list(map(lambda keyword: sorted(key_words.get_n_grams(keyword)),keywords))
students_candidates = list(map(lambda n_grams:list(map(lambda doc:key_words.get_candidates(n_grams, doc),docs[:20])),students_n_grams))

In [54]:
u = list(map(lambda st: list(map(len, st)),students_candidates))

In [57]:
len(u), len(u[0])

(2, 20, 20)

In [107]:
len(students_candidates[1])

20

In [59]:
t1 = time.perf_counter()
# students_candidates_emb = list(map(lambda cand:list(map(lambda x: np.array(
#     list(map(BERT.model.encode, x)))), cand)),students_candidates[0])
# students_candidates_emb = list(map(lambda st: list(map(emb_keywords, st)),students_candidates))
students_candidates_emb = list(map(lambda st: list(map(lambda kwrds: list(map(emb_keywords, kwrds)), st)) ,students_candidates))

print(time.perf_counter() - t1)
# keywords_embeddings =  list(map(lambda x: np.array(list(map(BERT.model.encode, x))), keywords))
# print(students_candidates_emb)
# print(keywords_embeddings[0].shape)

65.8284235000001


In [152]:
def kw_match(keywords_emb,candidates_emb):
    combination = list(zip(keywords_emb,candidates_emb))

    similarities = list(map(lambda comb:
                    np.array(cos_sim(comb[0],
                    comb[1])).clip(-1, 1).round(6),
                    combination))

    def fn_ (x: np.array):
        """
        return the no. of matched keywords
        """
        if not np.sum(x>= thershold):
            return 0
        if np.sum(x >= thershold) > 1.0:
            return 1.0
        return np.sum(x >= thershold)

    # res = np.sum(np.array(list(map(fn_, similarities))))
    # 5.51 ms ± 59.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) for 600 elements

    res = sum(map(fn_, similarities))
    # 5.45 ms ± 52.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) for 600 elements
    # return res/float(len(keywords_emb))
    return res/len(keywords_emb)

In [162]:
students_candidates_emb[0][0][0].shape

(40, 768)

In [262]:
model_candidates = list(map(lambda ans: key_words.candidates_tokens(ans,n_gram_range=(1,2)),ess_1_model_answers))
# model_candidate_emb = BERT.model.encode(model_candidates)
# emb_dict[ess_1_model_answers_emb]
model_candidate_emb = list(map(lambda cand:BERT.model.encode(cand),model_candidates))

In [183]:
# keywords = list(map(lambda x: maximal_marginal_relevance(
#     x[0].reshape(1, -1),x[1],x[2]),
#     zip(emb_dict['ess_1_model_answers_emb'][:2],
#     model_candidate_emb[:2],model_candidates[:2])))

In [263]:
keywords = list(map(lambda x: maximal_marginal_relevance(
    x[0].reshape(1, -1),x[1],x[2]),zip(emb_dict['ess_1_model_answers_emb'],
    model_candidate_emb,model_candidates)))

In [265]:
len(keywords), len(docs)

(22, 1672)

In [277]:
n_grams_list = []
students_candidates_list = []
keywords_embeddings_list = []
students_candidates_emb_list = []

batch = 50
n_docs = len(docs)
# all model answers
for ind, ans in enumerate(keywords):
    print(ind)
    students_n_grams = key_words.get_n_grams(ans)
    keywords_embeddings =  list(map(emb_keywords, ans))
    
    students_candidates_list_s = []
    students_candidates_emb_list_s = []
    # do in batches
    for i in range(0,n_docs,batch):
        students_candidates = list(map(lambda doc:
                        key_words.get_candidates(students_n_grams, doc),
                        docs[i:i+batch]))
        students_candidates_emb =  list(map(emb_keywords, students_candidates))
        students_candidates_list_s.extend(students_candidates)
        students_candidates_emb_list_s.extend(students_candidates_emb)

    if n_docs % batch != 0:
        students_candidates = list(map(lambda doc:
                        key_words.get_candidates(students_n_grams, doc),
                        docs[i+batch:]))
        students_candidates_emb =  list(map(emb_keywords, students_candidates))

        students_candidates_list_s.extend(students_candidates)
        students_candidates_emb_list_s.extend(students_candidates_emb)

    n_grams_list.append(students_n_grams)
    keywords_embeddings_list.append(keywords_embeddings)
    
    students_candidates_list.append(students_candidates_list_s)
    students_candidates_emb_list.append(students_candidates_emb_list_s)

0


C:\Users\oeb\AppData\Local\Temp/ipykernel_15544/1829135151.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(list(map(BERT.model.encode, keywords)))


Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
1
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
Error: empty vocabulary; perhaps the documents only contain stop words
2
Er

In [287]:
len(students_candidates_list[0]) , len(students_candidates_list[0][0]), len(students_candidates_list[0][0][0])

(1694, 2, 40)

*the length is abit missed up do we will ignore the last 50 as it is very expensive to re-compute*

In [289]:
# save the data
save_obj(keywords, 'data/essay1_keywords')
save_obj(n_grams_list, 'data/essay1_n_grams_list')
save_obj(keywords_embeddings_list, 'data/essay1_keywords_embeddings_list')
save_obj(students_candidates_list, 'data/essay1_students_candidates_list')
# this one is huge 5.5 GB
save_obj(students_candidates_emb_list, 'data/essay1_students_candidates_emb_list')

In [ ]:
keywords_grades_list = []
for i in range(len(n_grams_list)):
    print(i)
    keywords_grades = np.array(list(map(lambda st_cand:
                key_words.match_keywords(keywords_embeddings_list[i], st_cand,
                thershold=0.5),
                students_candidates_emb_list[i]
                )))
    keywords_grades_list.append(keywords_grades)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


# Optimzation

see what is faster

In [319]:
%timeit list(map(lambda doc:key_words.get_candidates(students_n_grams, doc),docs[:100]))

43.1 ms ± 319 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [320]:
docs_100 = docs[:100]
batch = 50
n_docs = len(docs_100)
def dummy():
    for i in range(0,n_docs,batch):
        students_candidates = list(map(lambda doc:
                    key_words.get_candidates(students_n_grams, doc),
                    docs_100[i:i+batch]))
%timeit dummy()

43 ms ± 363 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


> in batch mode is faster

In [313]:
#before 
n_grams_list


[[(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(2, 3)],
 [(1, 2), (2, 3)],
 [(2, 3)],
 [(2, 3)],
 [(2, 3)],
 [(2, 3)],
 [(1, 2), (2, 3)],
 [(2, 3)],
 [(1, 2), (2, 3)],
 [(1, 2), (2, 3)],
 [(2, 3)],
 [(2, 3)],
 [(2, 3)],
 [(2, 3)],
 [(2, 3)]]

In [370]:
batch = 50
# ! 
# n_docs = len(docs)
n_docs = 3

# we will try to eliminate redundant n_grams in further keyword extraction
# to omptimize the process
# dict comprehension
# ! becarful to double the len of the dict as mapping is can re 
n_gram_dict = {ind: {n_gram: None for n_gram in set([item for sublist in n_grams_list for item in sublist])} for ind ,doc in enumerate(docs[:n_docs+batch])}
# print(n_gram_dict)
# do the same process of keywords but using n_gram_dict
n_grams_list_x = []
students_candidates_list_x = []
keywords_embeddings_list_x = []
students_candidates_emb_list_x = []
def check_n_gram_dict(n_grams, doc, ind):
    res = []
    for n_gram in n_grams:
        if n_gram_dict[ind].get(n_gram) is None:
            # print("still not in dict",ind)
            x =  key_words.get_candidates([n_gram], doc)
            n_gram_dict[ind][n_gram] = x
            res.append(x)
        else:
            print("already in dict", ind)
            res.append(n_gram_dict[ind].get(n_gram))
    return res

if n_docs < batch:
    batch = n_docs

# all model answers
for ind, ans in enumerate(keywords[:2]):
    print(ind)
    students_n_grams = key_words.get_n_grams(ans)
    keywords_embeddings =  list(map(emb_keywords, ans))

    students_candidates_list_s_x = []
    students_candidates_emb_list_s_x = []

    # do in batches
    for i in range(0,n_docs,batch):
        # students_candidates = []
        # for ind, doc in enumerate(docs[i:i+batch]):
        #     students_candidates.extend(check_n_gram_dict(students_n_grams, doc, ind))
        students_candidates = list(map(lambda doc:
                        check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                        enumerate(docs[i:i+batch])))
        students_candidates_emb =  list(map(emb_keywords, students_candidates))

        students_candidates_list_s_x.extend(students_candidates)
        students_candidates_emb_list_s_x.extend(students_candidates_emb)

    if n_docs % batch != 0 and n_docs > batch:
        students_candidates = list(map(lambda doc:
                        key_words.get_candidates(students_n_grams, doc),
                        docs[i+batch:]))
        students_candidates_emb =  list(map(emb_keywords, students_candidates))

        students_candidates_list_s_x.extend(students_candidates)
        students_candidates_emb_list_s_x.extend(students_candidates_emb)

    n_grams_list_x.append(students_n_grams)
    keywords_embeddings_list_x.append(keywords_embeddings)

    students_candidates_list_x.append(students_candidates_list_s_x)
    students_candidates_emb_list_x.append(students_candidates_emb_list_s_x)

0
1
already in dict 0
already in dict 0
already in dict 1
already in dict 1
already in dict 2
already in dict 2
already in dict 3
already in dict 3
already in dict 4
already in dict 4
already in dict 5
already in dict 5
already in dict 6
already in dict 6
already in dict 7
already in dict 7
already in dict 8
already in dict 8
already in dict 9
already in dict 9
already in dict 10
already in dict 10
already in dict 11
already in dict 11
already in dict 12
already in dict 12
already in dict 13
already in dict 13
already in dict 14
already in dict 14
already in dict 15
already in dict 15
already in dict 16
already in dict 16
already in dict 17
already in dict 17
already in dict 18
already in dict 18
already in dict 19
already in dict 19
already in dict 20
already in dict 20
already in dict 21
already in dict 21
already in dict 22
already in dict 22
already in dict 23
already in dict 23
already in dict 24
already in dict 24
already in dict 25
already in dict 25
already in dict 26
already i

In [371]:
len(students_candidates_list_s_x),len(students_candidates_list_s_x[0]),len(students_candidates_list_s_x[0][0]),len(students_candidates_list_s_x[0][0][0]),

(50, 2, 1, 40)

In [377]:
keywords_grades_list_x = []
for i in range(len(keywords[:2])):
    print(i)
    keywords_grades = np.array(list(map(lambda st_cand:
                key_words.match_keywords(keywords_embeddings_list_x[i], st_cand,
                thershold=0.5),
                students_candidates_emb_list_x[i]
                )))
    keywords_grades_list_x.append(keywords_grades)

0
1


In [382]:
keywords_grades_list_x[0].shape[0] ==batch , len(keywords_grades_list_x) ==2

(True, True)

> same as expected

In [195]:
students_candidates_emb[0][0].shape ,students_candidates_emb[0].shape

((40, 768), (2,))

In [196]:
students_candidates_emb

[array([array([[ 2.1295432e-02,  1.7679781e-04,  2.1370557e-01, ...,
                -4.7906876e-02,  1.6631469e-01, -1.3477266e-01],
               [ 3.4837324e-02,  4.1363299e-01,  3.4743777e-01, ...,
                -2.3453319e-02, -1.5393102e-01, -1.4185250e-01],
               [ 5.0341442e-02,  1.5775688e-01,  3.1891915e-01, ...,
                -6.2302817e-02, -9.7999850e-04, -1.2756318e-01],
               ...,
               [ 7.9539068e-02, -1.4421634e-01,  2.3853749e-01, ...,
                 8.1818052e-02, -8.8140994e-02,  2.5620740e-02],
               [ 4.3749958e-02, -1.5811451e-01,  1.3444963e-02, ...,
                -2.0325319e-03,  6.9406003e-02, -6.1969459e-02],
               [ 1.2147342e-01,  2.7818733e-01,  3.0864418e-01, ...,
                 5.2360082e-01, -1.6034922e-01, -1.0349195e-01]], dtype=float32),
        array([[-1.52700648e-01, -2.33939305e-01,  3.17007661e-01, ...,
                 2.50961781e-01,  2.79449254e-01, -1.25926390e-01],
               [ 1.

# output essay 2

In [22]:
model_candidates = list(map(lambda ans: key_words.candidates_tokens(ans,n_gram_range=(1,2)),ess_2_model_answers))
model_candidate_emb = list(map(lambda cand:BERT.model.encode(cand),model_candidates))
keywords = list(map(lambda x: maximal_marginal_relevance(
    x[0].reshape(1, -1),x[1],x[2]),zip(emb_dict['ess_2_model_answers_emb'],
    model_candidate_emb,model_candidates)))
students_n_grams = list(map(lambda keyword: sorted(key_words.get_n_grams(keyword)),keywords))
docs = df.query('EssaySet == 2')["EssayText"].values.tolist()

In [125]:
key_words.candidates_tokens(str(docs[0]), n_gram_range=(1,2))

['improve',
 'used experiment',
 'student',
 'data names',
 'experiment plastic',
 'better new',
 'conclusion plastic',
 'type plastic',
 'student experiment',
 'types',
 'held stronger',
 'plastic length',
 'names',
 'data',
 'stronger different',
 'length conclusion',
 'experiment better',
 'grafin',
 'type grafin',
 'names type',
 'new data',
 'better',
 'experiment',
 'plastic type',
 'grafin improve',
 'new',
 'held',
 'conclusion',
 'changing type',
 'type used',
 'used',
 'length',
 'type held',
 'different',
 'improve student',
 'type',
 'changing',
 'different types',
 'plastic',
 'stronger']

In [127]:
get_candidates([(1,2),], docs[0])

[['improve',
  'used experiment',
  'student',
  'data names',
  'experiment plastic',
  'better new',
  'conclusion plastic',
  'type plastic',
  'student experiment',
  'types',
  'held stronger',
  'plastic length',
  'names',
  'data',
  'stronger different',
  'length conclusion',
  'experiment better',
  'grafin',
  'type grafin',
  'names type',
  'new data',
  'better',
  'experiment',
  'plastic type',
  'grafin improve',
  'new',
  'held',
  'conclusion',
  'changing type',
  'type used',
  'used',
  'length',
  'type held',
  'different',
  'improve student',
  'type',
  'changing',
  'different types',
  'plastic',
  'stronger']]

In [288]:
[ans for ind, ans in enumerate(keywords[:2])]

[['stretchability polymer',
  'did',
  'samples start',
  'measurements length',
  'plastic sample'],
 ['sample stretchability',
  'polymer plastics',
  'did',
  'samples start',
  'compared']]

In [346]:
keywords_embeddings_lis = []
for ind, ans in enumerate(keywords[:2]):
    print("iter",ind)
    keywords_embeddings =  np.array(list(map(BERT.model.encode, ans)))
    print(ans)
    keywords_embeddings_lis.append(keywords_embeddings)

iter 0
['stretchability polymer', 'did', 'samples start', 'measurements length', 'plastic sample']
iter 1
['sample stretchability', 'polymer plastics', 'did', 'samples start', 'compared']


In [336]:
emb_keywords([keywords[0]]).shape

(1, 5, 768)

In [347]:
len(keywords_embeddings_lis)

2

In [348]:
keywords_embeddings_lis[1].shape

(5, 768)

In [9]:
def get_candidates(n_grams, doc):
    x = list(map(lambda gram :
        key_words.candidates_tokens(str(doc), n_gram_range=gram)
        , n_grams))
    # print shapes
    # print([len(i) for i in x])
    return x

def keywords_pipeline_output(keywords,docs,n_gram_list,batch = 50):
    """
    Args:
        keywords: list of list of keywords
        docs: list of list of tokens
        batch: batch size
    Returns:
        a dict of each
            n_gram_dict : dict of dict of list of list of tokens
            n_grams_list : list of list of n_grams
            students_candidates_list : list of list of list of tokens
            keywords_embeddings_list  : list of list of list of embeddings
            students_candidates_emb_list : list of list of list of embeddings
            times : list of times taken for each model answer
    """

    def check_n_gram_dict(n_grams, doc, ind):
        res = []
        for n_gram in n_grams:
            x = n_gram_dict[ind].get(n_gram)
            if x is None:
                # x =  key_words.get_candidates(n_gram, doc)
                x =  get_candidates([n_gram], doc)
                # print("in",x)
                n_gram_dict[ind][n_gram] = x
                res.append(x)
            else:
                res.append(x)
        # print("out",res)
        return res


    def new_check_n_gram_dict(n_grams, doc, ind):
        # res = []
        # emb = []
        # for n_gram in n_grams:
        #     x = n_gram_dict[ind].get(n_gram)
        #     if x is None:
        #         # x =  key_words.get_candidates(n_gram, doc)
        #         x =  get_candidates([n_gram], doc)
        #         n_gram_dict[ind][n_gram] = x
        #         # cand_emb_dict[ind][n_gram] = list(map(BERT.model.encode, x))
        #         cand_emb_dict[ind][n_gram] = emb_keywords(x)
        #         # emb.append(cand_emb_dict[ind][n_gram])
        #         res.append(x)
        #     else:
        #         res.append(x)
        # return res
        def fn(n_gram):
            x = n_gram_dict[ind].get(n_gram)
            if x is None:
                # print("hi",ind)
                x =  get_candidates([n_gram], doc)
                n_gram_dict[ind][n_gram] = x
                cand_emb_dict[ind][n_gram] = emb_keywords(x)
                return x
            else:
                # print("f",ind)
                return x
        return list(map(fn, n_grams))
        # do it in parallel
        # return list(map(lambda n_gram:
        #     n_gram_dict[ind].get(n_gram)
        #     if n_gram_dict[ind].get(n_gram) is None
        #     else n_gram_dict[ind][n_gram], n_grams))


    def check_candidates_dict(n_grams, doc, ind):
        # res = []
        # for n_gram in n_grams:
        #     x = cand_emb_dict[ind].get(n_gram)
        #     if x is None:
        #         # x =  key_words.get_candidates(n_gram, doc)
        #         x =  get_candidates([n_gram], doc)
        #         cand_emb_dict[ind][n_gram] = x
        #         res.append(x)
        #     else:
        #         res.append(x)
        # return res
        # do it in parallel
        return list(map(lambda n_gram:
            cand_emb_dict[ind].get(n_gram)
            if cand_emb_dict[ind].get(n_gram) is None
            else cand_emb_dict[ind][n_gram],
            n_grams))



    n_docs = len(docs)
    if n_docs < batch:
        batch = n_docs
    
    n_gram_dict = {ind: {n_gram: None for n_gram in set([item for sublist in n_gram_list for item in sublist])} for ind ,doc in enumerate(docs)}
    cand_emb_dict = {ind: {n_gram: None for n_gram in set([item for sublist in n_gram_list for item in sublist])} for ind ,doc in enumerate(docs)}

    n_grams_list = []
    students_candidates_list = []
    keywords_embeddings_list = []
    students_candidates_emb_list = []
    times = []
    hh_list = []

    # all model answers
    for ind, ans in enumerate(keywords):
        print("model answer",ind)
        t1 = time.perf_counter()
        students_n_grams = key_words.get_n_grams(ans)
        keywords_embeddings =  list(map(BERT.model.encode, ans))

        students_candidates_list_s = []
        students_candidates_emb_list_s = []
        # hh_list_s = []

        # do in batches
        for i in range(0,n_docs,batch):
            # students_candidates = [0]
            # students_candidates = list(map(lambda doc:
            #                 check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
            #                 enumerate(docs[i:i+batch])))
            # print("students_candidates",students_candidates[0])
            # students_candidates_emb = []
            # for j,k in enumerate(students_candidates):
            #     print(j,k)
            #     students_candidates_emb.extend(emb_keywords(k))
            # print("students_candidates",len(students_candidates))
            # students_candidates_emb =  []
            students_candidates = list(map(lambda doc:
                            new_check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i:i+batch])))
            students_candidates_emb = list(map(lambda doc:
                            check_candidates_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i:i+batch])))
            # students_candidates_emb =  list(map(lambda st: list(map( emb_keywords, st)),students_candidates))
            # students_candidates_emb =  list(map(lambda doc:
            #                 check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
            #                 enumerate(docs[i:i+batch])))
            # hh = [x.shape for x in students_candidates_emb]
            # students_candidates_emb =  list(map(fn, students_candidates))
            # hh =  list(map(old_fn, students_candidates))
            # list(map(lambda doc:
            #                 check_candidates_dict(students_n_grams, doc[1], i+doc[0]),
            #                 enumerate(students_candidates)))

            students_candidates_list_s.extend(students_candidates)
            # hh_list_s.extend(hh)
            students_candidates_emb_list_s.extend(students_candidates_emb)

        if n_docs % batch != 0 and n_docs > batch:
            students_candidates = list(map(lambda doc:
                            check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i+batch:])))
            students_candidates_emb =  list(map(emb_keywords, students_candidates))

            students_candidates_list_s.extend(students_candidates)
            students_candidates_emb_list_s.extend(students_candidates_emb)

        n_grams_list.append(students_n_grams)
        keywords_embeddings_list.append(keywords_embeddings)

        students_candidates_list.append(students_candidates_list_s)
        students_candidates_emb_list.append(students_candidates_emb_list_s)
        # hh_list.append(hh_list_s)
        times.append(time.perf_counter() - t1)
    # retun a dict of each n_grams_list, students_candidates_list, keywords_embeddings_list, students_candidates_emb_list
    return {
        "n_gram_dict": n_gram_dict,
        "n_grams_list": n_grams_list,
        # "cand_emb_dict": cand_emb_dict,
        # "hh_list": hh_list,
        "students_candidates_list": students_candidates_list,
        "keywords_embeddings_list": keywords_embeddings_list,
        "students_candidates_emb_list": students_candidates_emb_list,
        "times": times
    }


In [10]:
def match_keywords(
    keywords_emb:list[np.ndarray],
    candidates_emb : list[np.ndarray],
    thershold: float)\
        -> float:
    """
    match keywords with candidates in a document

    Args:
        keywords_emb (List[np.ndarray]): list of keywords embeddings
        candidates_emb (List[np.ndarray]): list of document's candidates embeddings
        thershold (float): threshold

    Returns:
        float: score
    
    example:
        >>> match_keywords(keywords_emb, candidates_emb, thershold=0.5)
        >>> 0.8
    """
    # candidates_emb_shapes = list(map(lambda emb: emb.shape, candidates_emb))
    # keys_emb_shapes = list(map(lambda emb: emb.reshape(1, -1).shape, keywords_emb))
    # check if candidates_emb shape is 1d
    # def shape_check(emb):
    #     if emb.size == 1:
    #         return emb.reshape(-1, 1)
    #     else:
    #         return emb
    
    # candidates_emb = list(map(shape_check, candidates_emb))
    # keywords_emb = list(map(shape_check, keywords_emb))

    # combination = list(zip(keywords_emb,candidates_emb))
    # print("checked")

    # similarities = list(map(lambda comb:
    #                 np.array(cos_sim(comb[0],
    #                 comb[1])).clip(-1, 1).round(6),
    #                 combination))


    # similarities = list(map(lambda comb:
    #                 print(comb[0].shape,comb[1].shape),
    #                 combination))

    # similarities = list(map(lambda cand:
    #                 print(np.array(keywords_emb).shape, cand.reshape(cand.shape[1],cand.shape[2]).shape),
    #                 candidates_emb))
    # .__array__().max()

    # similarities = list(map(lambda cand:
    #                 print(np.array(keywords_emb).shape, cand.shape),
    #                 candidates_emb))

    similarities = list(map(lambda cand:
                    cos_sim(np.array(keywords_emb), cand.reshape(cand.shape[0],cand.shape[1])).__array__().max(axis=1).round(6).clip(-1, 1),
                    candidates_emb))
    # print shape of similarities
    # print(np.array(similarities).shape)



    # similarities = list(map(lambda cand:
    #                 np.array(list(map(lambda key:
    #                 cos_sim(key, cand.reshape(cand.shape[1],cand.shape[2])).clip(-1, 1).round(6),keywords_emb))),
    #                 # print(key.shape, cand.reshape(cand.shape[1],cand.shape[2]).shape),keywords_emb))),
    #                 candidates_emb))

    # def fn_ (x: np.array):
    #     """
    #     return the no. of matched keywords
    #     """
    #     if not np.sum(x>= thershold):
    #         return 0
    #     if np.sum(x >= thershold) > 1.0:
    #         return 1.0
    #     return np.sum(x >= thershold)

    # res = sum(map(fn_, similarities))
    # return res/len(keywords_emb)
    # print(type(combination[0][1]))

    # similarities = list(map(lambda comb:
    #                 list(map(lambda comb_:
    #                 print(type(comb[0]),type(comb_)),comb[1])),
    #                 combination))

    # similarities = list(map(lambda comb:
    #                 print(type(comb[0]),type(comb[1])),
    #                 combination))
    # list(map(lambda sim: print((sim.__array__().shape)) , similarities))
    return similarities

In [11]:
def para_keywords_pipeline_output(keywords,docs,n_gram_list,batch = 50):

    def check_n_gram_dict(n_grams, doc, ind):
        res = []
        for n_gram in n_grams:
            x = n_gram_dict[ind].get(n_gram)
            if x is None:
                x =  get_candidates([n_gram], doc)
                n_gram_dict[ind][n_gram] = x
                res.append(x)
            else:
                res.append(x)
        return res


    def new_check_n_gram_dict(n_grams, doc, ind):
        def fn(n_gram):
            x = n_gram_dict[ind].get(n_gram)
            if x is None:
                # print("hi",ind)
                x =  get_candidates([n_gram], doc)
                n_gram_dict[ind][n_gram] = x
                # cand_emb_dict[ind][n_gram] = emb_keywords(x)
                return x
            else:
                # print("f",ind)
                return x
        return list(map(fn, n_grams))


    def check_candidates_dict(n_grams, doc, ind):
        def fn(n_gram):
            x = cand_emb_dict[ind].get(n_gram)
            if x is None:
                cand_emb_dict[ind][n_gram] = emb_keywords(n_gram_dict[ind][n_gram])
                return cand_emb_dict[ind][n_gram]
            else:
                return x
        return list(map(fn,n_grams))

    n_docs = len(docs)
    if n_docs < batch:
        batch = n_docs
    
    n_gram_dict = {ind: {n_gram: None for n_gram in set([item for sublist in n_gram_list for item in sublist])} for ind ,doc in enumerate(docs)}
    cand_emb_dict = {ind: {n_gram: None for n_gram in set([item for sublist in n_gram_list for item in sublist])} for ind ,doc in enumerate(docs)}

    n_grams_list = []
    students_candidates_list = []
    keywords_embeddings_list = []
    students_candidates_emb_list = []
    times = []
    hh_list = []

    # all model answers
    for ind, ans in enumerate(keywords):
        print("model answer",ind)
        t1 = time.perf_counter()
        students_n_grams = key_words.get_n_grams(ans)
        keywords_embeddings =  list(map(BERT.model.encode, ans))

        students_candidates_list_s = []
        students_candidates_emb_list_s = []
        # hh_list_s = []

        # do in batches
        for i in range(0,n_docs,batch):
            students_candidates = [0]

            students_candidates = list(map(lambda doc:
                            new_check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i:i+batch])))
            students_candidates_emb = list(map(lambda doc:
                            check_candidates_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i:i+batch])))
            students_candidates_list_s.extend(students_candidates)
            # hh_list_s.extend(hh)
            students_candidates_emb_list_s.extend(students_candidates_emb)

        if n_docs % batch != 0 and n_docs > batch:
            students_candidates = list(map(lambda doc:
                            check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i+batch:])))
            # students_candidates_emb = list(map(lambda doc:
            #                 check_candidates_dict(students_n_grams, doc[1], i+doc[0]),
            #                 enumerate(docs[i:i+batch])))
            students_candidates_emb =  list(map(emb_keywords, students_candidates))

            students_candidates_list_s.extend(students_candidates)
            students_candidates_emb_list_s.extend(students_candidates_emb)

        n_grams_list.append(students_n_grams)
        keywords_embeddings_list.append(keywords_embeddings)

        students_candidates_list.append(students_candidates_list_s)
        students_candidates_emb_list.append(students_candidates_emb_list_s)
        # hh_list.append(hh_list_s)
        times.append(time.perf_counter() - t1)
    # retun a dict of each n_grams_list, students_candidates_list, keywords_embeddings_list, students_candidates_emb_list
    return {
        "n_gram_dict": n_gram_dict,
        "n_grams_list": n_grams_list,
        "students_candidates_list": students_candidates_list,
        "keywords_embeddings_list": keywords_embeddings_list,
        "students_candidates_emb_list": students_candidates_emb_list,
        "times": times
    }
    

In [12]:
def para_keywords_pipeline_output_dict(keywords,docs,n_gram_list,batch = 50):

    def check_n_gram_dict(n_grams, doc, ind):
        def fn(n_gram):
            x = n_gram_dict[ind].get(n_gram)
            if x is None:
                x =  get_candidates([n_gram], doc)
                n_gram_dict[ind][n_gram] = x
                return x
            else:
                return x
        return list(map(fn, n_grams))

    n_docs = len(docs)
    if n_docs < batch:
        batch = n_docs
    
    n_gram_dict = {ind: {n_gram: None for n_gram in set([item for sublist in n_gram_list for item in sublist])} for ind ,doc in enumerate(docs)}

    n_grams_list = []
    students_candidates_list = []
    keywords_embeddings_list = []
    students_candidates_emb_list = []
    times = []

    # all model answers
    for ind, ans in enumerate(keywords):
        print("model answer",ind)
        t1 = time.perf_counter()
        students_n_grams = key_words.get_n_grams(ans)
        keywords_embeddings =  list(map(BERT.model.encode, ans))

        students_candidates_list_s = []
        students_candidates_emb_list_s = []
        # do in batches
        for i in range(0,n_docs,batch):
            students_candidates = list(map(lambda doc:
                            check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i:i+batch])))
            students_candidates_emb =  list(map( lambda st: list(map( emb_keywords ,st)), students_candidates))
            # students_candidates_emb =  list(map(emb_d, students_candidates))
            students_candidates_list_s.extend(students_candidates)
            students_candidates_emb_list_s.extend(students_candidates_emb)

        if n_docs % batch != 0 and n_docs > batch:
            students_candidates = list(map(lambda doc:
                            check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
                            enumerate(docs[i+batch:])))
            students_candidates_emb =  list(map( lambda st: list(map( emb_keywords ,st)), students_candidates))

            students_candidates_list_s.extend(students_candidates)
            students_candidates_emb_list_s.extend(students_candidates_emb)

        n_grams_list.append(students_n_grams)
        keywords_embeddings_list.append(keywords_embeddings)
        students_candidates_list.append(students_candidates_list_s)
        students_candidates_emb_list.append(students_candidates_emb_list_s)
        times.append(time.perf_counter() - t1)
    return {
        "n_gram_dict": n_gram_dict,
        "n_grams_list": n_grams_list,
        "students_candidates_list": students_candidates_list,
        "keywords_embeddings_list": keywords_embeddings_list,
        "students_candidates_emb_list": students_candidates_emb_list,
        "times": times
    }


In [13]:
def emb_(keywords):
    # x = list(map(lambda st: list(map(lambda k:
    #  list(map(lambda ans: len(ans),k)) ,st)) ,st_cands))
    #! x = list(map(lambda k:list(map(lambda x: BERT.model.encode(str(x)) ,k)) ,keywords))
    x = list(map(lambda k:list(map(lambda x: get_words_emb(str(x)) ,k)) ,keywords))
    # x = np.array(list(map(lambda k:
    #  np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),st_cands)))
    # if x.ndim == 3:
    #     x = x.reshape(max(x.shape[1],x.shape[0]),x.shape[2])
    # print("emb",x.shape)
    return x
    # return list(map(lambda k: np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords))

In [14]:
def normal_keywords_pipeline_output(keywords,docs,n_gram_list,batch = 50):
    # if not isinstance(docs, list):
    #     docs = [docs]

    n_docs = len(docs)
    if n_docs < batch:
        batch = n_docs
    keywords_embeddings_list = []
    students_candidates_emb_list = []

    students_n_grams = key_words.get_n_grams(keywords)
    keywords_embeddings =  list(map(BERT.model.encode, keywords))
    #! keywords_embeddings =  list(map(BERT.model.encode, [keywords]))
    # print("keywords_embeddings",len(keywords_embeddings))
    # print("keywords_embeddings",keywords_embeddings[0].shape)

    students_candidates_emb_list_s = []
    # do in batches
    print("students n grams",students_n_grams)
    for i in range(0,n_docs,batch):
        students_candidates = list(map(lambda doc: get_candidates(students_n_grams, doc),docs[i:i+batch]))
        # print  lens of students_candidates
        print("students_candidates",[[len(y) for y in st ]for st in students_candidates])
        # students_candidates = list(map(lambda doc:
        #                 new_check_n_gram_dict(students_n_grams, doc[1], i+doc[0]),
        #                 enumerate(docs[i:i+batch])))

        # students_candidates_emb = emb_keywords(n_gram_dict[ind][n_gram])
        # list(map(lambda n_grams:list(map(lambda doc:
        #     key_words.get_candidates(n_grams, doc),docs[i:i+batch])),students_n_grams))

        # students_candidates_emb =  list(map(BERT.model.encode, students_candidates))
        # students_candidates_emb =  list(map(lambda st: list(map( lambda x: BERT.model.encode(str(x)), st)),students_candidates))
        # students_candidates_emb = list(map(lambda st: list(map( lambda x: len(x), st)),students_candidates))
        # print(list(map(lambda st: list(map( lambda x: len(x), st)),students_candidates)))
        # print(list(map(lambda st: emb_keywords(st).shape,students_candidates)))
        students_candidates_emb =  list(map( emb_, students_candidates))
        # print(emb_keywords(students_candidates).shape)
        # students_candidates_emb = list(map(lambda doc:
        #                 check_candidates_dict(students_n_grams, doc[1], i+doc[0]),
        #                 enumerate(docs[i:i+batch])))
        students_candidates_emb_list_s.extend(students_candidates_emb)

    if n_docs % batch != 0 and n_docs > batch:
        students_candidates = list(map(lambda doc:
            get_candidates(students_n_grams, doc),docs[i:i+batch]))
        # students_candidates_emb =  list(map( lambda st: list(map(emb_keywords, st)),students_candidates))
        students_candidates_emb =  list(map(lambda st: list(map( lambda x: BERT.model.encode(str(x)), st)),students_candidates))
        students_candidates_emb_list_s.extend(students_candidates_emb)

    keywords_embeddings_list.append(keywords_embeddings)
    students_candidates_emb_list.append(students_candidates_emb_list_s)
    return {
        "keywords_embeddings_list": keywords_embeddings_list,
        "students_candidates_emb_list": students_candidates_emb_list,
    }

In [47]:
def emb_keywords(keywords):
    # x = np.array(list(map(lambda k: np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords)))
    x = np.array(list(map(lambda k: np.array(list(map(lambda x: get_words_emb(str(x)), k))),keywords)))
    # print("emb be",x.shape)

    if x.ndim == 3:
        x = x.reshape(max(x.shape[1],x.shape[0]),x.shape[2])
    # print("emb",x.shape)
    return x
    # return list(map(lambda k: np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords))

In [46]:
words_emb_dict = {}

# def emb_keywords(keywords):
#     # return np.array(list(map(lambda k: np.array(list(map(lambda x: BERT.model.encode(str(x)), k))),keywords)))
#     return np.array(list(map(lambda k: np.array(list(map(lambda x: get_words_emb(str(x)), k))),keywords)))
#     # return np.array(list(map(BERT.model.encode, keywords)))

def get_words_emb(word):
    if word in words_emb_dict:
        return words_emb_dict[word]
    else:
        words_emb_dict[word] = BERT.model.encode(word)
        return words_emb_dict[word]

def emb_d(keywords):
    return np.array(list(map(lambda k: np.array(list(map(lambda x: get_words_emb(str(x)) ,k))) ,keywords)))
    # return list(map(lambda k:list(map(lambda x: get_words_emb(str(x)) ,k)) ,keywords))

def normal_keywords_pipeline_output_with_dict(keywords,docs,n_gram_list,batch = 10):
    # if not isinstance(docs, list):
    #     docs = [docs]
    words_emb_dict = {}

    n_docs = len(docs)
    if n_docs < batch:
        batch = n_docs
    keywords_embeddings_list = []
    students_candidates_emb_list = []

    students_n_grams = key_words.get_n_grams(keywords)
    keywords_embeddings =  list(map(BERT.model.encode, keywords))
    #! keywords_embeddings =  list(map(BERT.model.encode, [keywords]))
    students_candidates_emb_list_s = []
    # do in batches
    # print("students n grams",students_n_grams)
    for i in range(0,n_docs,batch):
        students_candidates = list(map(lambda doc: get_candidates(students_n_grams, doc),docs[i:i+batch]))
        # print("students_candidates",[[len(y) for y in st ]for st in students_candidates])
        students_candidates_emb =  list(map( emb_d, students_candidates))
        # students_candidates_emb =  list(map( emb_d, students_candidates))
        students_candidates_emb_list_s.extend(students_candidates_emb)

    if n_docs % batch != 0 and n_docs > batch:
        students_candidates = list(map(lambda doc:
            get_candidates(students_n_grams, doc),docs[i:i+batch]))
        # students_candidates_emb =  list(map( lambda st: list(map(emb_keywords, st)),students_candidates))
        # students_candidates_emb =  list(map(lambda st: list(map( lambda x: BERT.model.encode(str(x)), st)),students_candidates))
        students_candidates_emb =  list(map( emb_d, students_candidates))
        students_candidates_emb_list_s.extend(students_candidates_emb)

    keywords_embeddings_list.append(keywords_embeddings)
    students_candidates_emb_list.append(students_candidates_emb_list_s)
    return {
        "keywords_embeddings_list": keywords_embeddings_list,
        "students_candidates_emb_list": students_candidates_emb_list,
    }

In [158]:
max(list(words_emb_dict.keys()))

'weights second way'

In [17]:
def grading(keywords_embeddings_list,students_candidates_emb_list,thershold=0.5):
    """
    Args:
        keywords_embeddings_list: list of list of list of embeddings
        students_candidates_emb_list: list of list of list of embeddings
        thershold: thershold for the similarity
    Returns:
        a list of list of list of grades
    """
    grades = []
    for i in range(len(keywords_embeddings_list)):
        # for j in range(len(keywords_embeddings_list[i])):
        #     grades.append(key_words.match_keywords(keywords_embeddings_list[i][j],
        #                                     students_candidates_emb_list[i][j],
        #                                     thershold))
        
        # map(lambda j: grades.append(key_words.match_keywords(keywords_embeddings_list[i][j],)))
        grades.append(np.array(list(map(lambda st_cand:
                match_keywords(keywords_embeddings_list[i], st_cand,
                thershold=thershold),
                students_candidates_emb_list[i]
                ))))
    grades = np.array(list(map(lambda sim: (sim.__array__().max(axis=1) >thershold).sum(axis=1)/float(sim.shape[-1]) , grades)))
    return grades

In [ ]:
# random array of shape (22, 1, 768) (22, 1, 768) (1, 40, 768)
random_array = np.random.rand(22, 1, 768)

In [394]:
cos_sim(np.random.rand(5,768), np.random.rand(40, 768)).__array__().max(axis=1)

array([0.77372219, 0.75673716, 0.78498689, 0.77073208, 0.76771135])

In [429]:
g = grading(keywords_embeddings_lis,d['students_candidates_emb_list'],0.5)

(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)
(2, 5)
(5,)
(5,)


In [433]:
g.shape

(2, 20)

In [83]:
model_candidates = list(map(lambda ans: key_words.candidates_tokens(ans,n_gram_range=(1,2)),ess_2_model_answers))
model_candidate_emb = list(map(lambda cand:BERT.model.encode(cand),model_candidates))
keywords = list(map(lambda x: maximal_marginal_relevance(
    x[0].reshape(1, -1),x[1],x[2]),zip(emb_dict['ess_2_model_answers_emb'],
    model_candidate_emb,model_candidates)))
students_n_grams = list(map(lambda keyword: sorted(key_words.get_n_grams(keyword)),keywords))
docs = df.query('EssaySet == 2')["EssayText"].values.tolist()

In [107]:
t1 = time.perf_counter()
dd = para_keywords_pipeline_output_dict(keywords[0:2],docs[:4],students_n_grams,batch=10)
time.perf_counter() - t1

model answer 0
emb be (1, 40, 768)
emb (40, 768)
emb be (1, 41, 768)
emb (41, 768)
emb be (1, 45, 768)
emb (45, 768)
emb be (1, 45, 768)
emb (45, 768)
emb be (1, 27, 768)
emb (27, 768)
emb be (1, 25, 768)
emb (25, 768)
emb be (1, 12, 768)
emb (12, 768)
emb be (1, 11, 768)
emb (11, 768)
model answer 1
emb be (1, 40, 768)
emb (40, 768)
emb be (1, 41, 768)
emb (41, 768)
emb be (1, 45, 768)
emb (45, 768)
emb be (1, 45, 768)
emb (45, 768)
emb be (1, 27, 768)
emb (27, 768)
emb be (1, 25, 768)
emb (25, 768)
emb be (1, 12, 768)
emb (12, 768)
emb be (1, 11, 768)
emb (11, 768)


0.41940510000131326

In [108]:
grading(dd['keywords_embeddings_list'],dd['students_candidates_emb_list'],0.5)

array([[1.        , 1.        , 1.        , 0.57142857],
       [1.        , 1.        , 0.85714286, 0.42857143]])

In [759]:
d = keywords_pipeline_output([keywords[0]],docs[:4],students_n_grams,batch=10)

model answer 0
[40]
emb (40, 768)
[44]
emb (44, 768)
[43]
emb (43, 768)
[45]
emb (45, 768)
[22]
emb (22, 768)
[21]
emb (21, 768)
[43]
emb (43, 768)
[44]
emb (44, 768)


In [678]:
%timeit keywords_pipeline_output([keywords[0]],docs[:5],students_n_grams,batch=10)

model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
10 s ± 233 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [679]:
d['times']

[7.98857470000803]

In [680]:
%timeit para_keywords_pipeline_output([keywords[0]],docs[:5],students_n_grams,batch=10)

model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
model answer 0
9.75 s ± 47.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [148]:
t1 = time.perf_counter()
dn = normal_keywords_pipeline_output(keywords[0],docs[:3],students_n_grams,batch=10)
time.perf_counter() - t1

students n grams [(1, 2), (2, 3)]
students_candidates [[40, 44], [43, 45], [22, 21]]


C:\Users\oeb\AppData\Local\Temp/ipykernel_15280/2889523460.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(list(map(lambda k:


5.166980099998909

In [89]:
grading(dn['keywords_embeddings_list'],dn['students_candidates_emb_list'],0.5)

array([[0.6, 0.8, 0.6]])

In [87]:
t1 = time.perf_counter()
dn = normal_keywords_pipeline_output_with_dict(keywords[0],docs[:3],students_n_grams,batch=10)
time.perf_counter() - t1

C:\Users\oeb\AppData\Local\Temp/ipykernel_15916/2509446158.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(list(map(lambda k: np.array(list(map(lambda x: get_words_emb(str(x)) ,k))) ,keywords)))


0.14998909999849275

In [118]:
# {"asndgv":[1,2,3,3]}*2
embdd = BERT.model.encode("asndgv")
dic = {}
dic = {"asndgv"+str(int(k)) : embdd for k in list(np.arange(1,1000*100))}

In [120]:
sys.getsizeof(dic) / 1024 / 1024

5.000083923339844

In [133]:
d1 = para_keywords_pipeline_output(keywords[:4],docs[:4],students_n_grams,batch=10)

model answer 0
model answer 1
model answer 2
model answer 3


In [164]:
d1['times']

[0.12975770000048215,
 0.12392139999974461,
 0.12380529999973078,
 0.12464680000084627]

In [540]:
"keywords_embeddings_list"
# n models
# [0] n of keywords ARRAY

'keywords_embeddings_list'

In [539]:
"students_candidates_emb_list"
# n models
# [0] n of students
# [0][0] each student's candidates (40, 768) , (41, 768) ARRAY

'students_candidates_emb_list'

In [81]:
for i, v in enumerate(dd['students_candidates_emb_list'][0]):
    try:
        print("arr",i, v.shape)
    except:
        print(i, len(v))

arr 0 (2, 768)
arr 1 (2, 768)
arr 2 (2, 768)
arr 3 (2, 768)


In [137]:
grading(d1['keywords_embeddings_list'],d1['students_candidates_emb_list'],0.5).max(axis=0)

ValueError: cannot reshape array of size 30720 into shape (1,40)

In [215]:
def fn(keywords):
    return list(map(lambda k: np.array(list(map(lambda x: np.array(list(map(lambda y:  BERT.model.encode(str(y)),x))), k))),keywords))
    # return np.array(list(map(BERT.model.encode, keywords)))
def fn_old(keywords):
    return list(map(lambda k: np.array(list(map(lambda x: list(map(lambda y:  BERT.model.encode(str(y)),x)), k))),keywords))
    # return[list(map(lambda x: list(map(lambda y: str(y)+" 1",x)), k)) for k in keywords]
    # return np.array(list(map(BERT.model.encode, keywords)))

In [183]:
d['times'] # it will be huge difference for bigger data

[23.139581300005375, 22.66242659999989, 22.939434699997946, 23.50101580000046]

In [177]:
n_gram_dict[49]

{(2,
  3): [['stretched minutes',
   'type way student',
   '12mm trial conclusion',
   'minutes stretching 22mm',
   'stretched minutes stretching',
   'stretched type',
   'type stretched 10mm',
   'weight added trials',
   'experiment tell type',
   'stretching 22mm',
   '10mm trial 12mm',
   '23mm trial',
   'student improve',
   'tell weight added',
   'type stretched type',
   'polymer type',
   'stretched 10mm trial',
   'way student improve',
   '12mm trial',
   'added trials',
   '23mm trial plastic',
   'conclusion strongest',
   'experiment tell',
   'trial conclusion',
   '10mm trial',
   'type way',
   'trial 12mm trial',
   'improve experiment',
   'type stretched',
   'stretched 10mm',
   'trial plastic type',
   'way student',
   'trial 23mm trial',
   'polymer type way',
   'conclusion strongest polymer',
   'plastic type',
   'strongest polymer type',
   'student improve experiment',
   'stretching 22mm trial',
   'tell type plastics',
   'minutes stretching',
   'typ

> **perfect now the full essays**

In [145]:
kwrds=[]
for k,st in zip(keywords_embeddings,students_candidates_emb[0]):
    print(k.shape,st[0].shape)
    # kwrds = key_words.match_keywords(k,st,thershold=0.5)
    # kwrds = np.array(list(map(lambda x: key_words.match_keywords(k,x,thershold=0.5),st)))
    kwrds = np.array(list(map(lambda x: x,st)))
# lambda st_cand:
#                     key_words.match_keywords(k, st_cand,
#                     thershold=.5)
    # print(len(st))

(5, 768) (40, 768)
(5, 768) (45, 768)


C:\Users\oeb\AppData\Local\Temp/ipykernel_15544/4174833780.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  kwrds = np.array(list(map(lambda x: x,st)))


In [146]:
model_candidates = key_words.candidates_tokens(docs[0],n_gram_range=n_gram_range)
model_candidate_emb = self.model.encode(model_candidates)


keywords_ = key_words.maximal_marginal_relevance(
        model_answer_emb, model_candidate_emb,
        model_candidates,
        top_n =top_n, diversity=diversity)

# keywords__emb = model.encode(keywords_)
students_n_grams = key_words.get_n_grams(keywords_)

students_candidates = list(map(lambda doc:
            key_words.get_candidates(students_n_grams, doc),
            docs[1:]))

students_candidates_emb = list(map(self.__emb_keywords, students_candidates))
keywords_embeddings =  list(map(self.__emb_keywords, keywords_))

keywords_grades = np.array(list(map(lambda st_cand:
            key_words.match_keywords(keywords_embeddings, st_cand,
            thershold=threshold),
            students_candidates_emb,
            )))

(2, 45, 768)

In [73]:
keywords_grades = np.array(list(map(
    lambda k_emb : np.array(list(map(lambda st_cand:
            key_words.match_keywords(k_emb, st_cand,
            thershold=.5),
            students_candidates_emb,
            ))),keywords_embeddings)))

AttributeError: 'list' object has no attribute 'size'

# moment of truth

In [18]:
import string
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [64]:
def get_ngram_top_n_diversity(docs):
    lens = []
    tokens = []
    for doc in docs:
        tokens_ = [w.translate(str.maketrans('', '', string.punctuation)) for w in [doc]]
        tokens_ = [w for w in set(doc.split()) if w not in stop_words]
        tokens.append(tokens_)
        lens.append(len(tokens))

    model_candidates = tokens
    model_candidate_emb = np.array(list(map(lambda cand:BERT.model.encode(cand),model_candidates)))
    print(model_candidate_emb.shape)
    # list(map(lambda st:cos_sim(model_answer_emb,st),model_candidate_emb))
    # cos_sim(model_answer_emb, model_candidate_emb)

    lens = np.median(np.array(lens))
    return lens

In [29]:
for i,v in ess_dict.items():
    l = get_ngram_top_n_diversity(v)
    print(l)


23.5
19.5
33.0


In [19]:
ess_dict = {
    "ess_1_model_answers": ess_1_model_answers,
    "ess_2_model_answers": ess_2_model_answers,
    "ess_3_model_answers": ess_3_model_answers,
    "ess_4_model_answers": ess_4_model_answers,
    "ess_5_model_answers": ess_5_model_answers,
    "ess_6_model_answers": ess_6_model_answers,
    "ess_7_model_answers": ess_7_model_answers,
    "ess_8_model_answers": ess_8_model_answers,
    "ess_9_model_answers": ess_9_model_answers,
    "ess_10_model_answers": ess_10_model_answers,
}

In [60]:
for i,v in ess_dict.items():
    l = get_ngram_top_n_diversity(v)
    if l < 10:
        # short 
        # 
        tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in [doc]]
        tokens = [w for w in set(doc.split()) if w not in stop_words]
        model_candidates = tokens
        model_candidate_emb = BERT.model.encode(model_candidates)
        cos_sim(model_answer_emb, model_candidate_emb)
    print(l)


19.0
19.0
30.0
34.0
37.0
29.5
20.5
32.0
35.0
25.5


In [61]:
def compare_keys_div(ind,essay,top_n=5,diversities=[0.8]):
    for div in diversities:
        keywords = list(map(lambda x: maximal_marginal_relevance(x[0].reshape(1, -1),x[1],x[2],top_n=top_n,diversity=div),
            zip(emb_dict[f'ess_{essay}_model_answers_emb'],
            model_candidate_emb,model_candidates)))
        print(div,keywords[ind])

In [20]:
# out = {}
top_n_l = [7]*10
diversity_l = [0.6,0.6, 0.7,0.7,0.7,0.7,0.6,0.7,0.7,0.7]
ngram_range_l = [(2,2),(2,2),(2,3),(2,3),(2,3),(2,3),(2,2),(2,3),(2,3),(2,3)]

In [86]:
for essay in range(0,10):
    # get top_n, diversity, ngram_range
    print("essayset",essay+1)
    
    model_candidates = list(map(lambda ans: key_words.candidates_tokens(ans,n_gram_range=ngram_range_l[essay+1]),ess_dict[f'ess_{essay+1}_model_answers']))
    model_candidate_emb = list(map(lambda cand:BERT.model.encode(cand),model_candidates))
    keywords = list(map(lambda x: maximal_marginal_relevance(
        x[0].reshape(1, -1),x[1],x[2],top_n=top_n_l[essay+1],diversity=diversity_l[essay+1]),
        zip(emb_dict[f'ess_{essay+1}_model_answers_emb'],
        model_candidate_emb,model_candidates)))
    students_n_grams = list(map(lambda keyword: sorted(key_words.get_n_grams(keyword)),keywords))

    docs = df.query(f'EssaySet == {essay+1}')["EssayText"].values.tolist()

    dd = para_keywords_pipeline_output_dict(keywords,docs,students_n_grams,batch=10)
    
    save_obj(dd,f'data/results/keywords_res_essay_{essay+1}')
    # out[f'ess_{essay+1}_keywords'] = dd
    # save_obj(out,f'data/results/keywords_output_dict{essay+1}')

essayset 1
model answer 0
model answer 1
model answer 2
model answer 3
model answer 4
model answer 5
model answer 6
model answer 7
model answer 8
model answer 9
model answer 10
model answer 11
model answer 12
model answer 13
model answer 14
model answer 15
model answer 16
model answer 17
model answer 18
model answer 19
model answer 20
model answer 21
essayset 2
model answer 0
model answer 1
model answer 2
model answer 3
model answer 4
model answer 5
model answer 6
model answer 7
model answer 8
model answer 9
model answer 10
model answer 11
model answer 12
model answer 13
model answer 14
model answer 15
model answer 16
model answer 17
model answer 18
model answer 19
model answer 20
model answer 21
essayset 3
model answer 0
model answer 1
model answer 2
model answer 3
model answer 4
model answer 5
model answer 6
model answer 7
model answer 8
model answer 9
model answer 10
model answer 11
model answer 12
model answer 13
model answer 14
model answer 15
model answer 16
model answer 17
model

KeyboardInterrupt: 

In [88]:
save_obj(words_emb_dict,f'data/results/words_emb_dict')

In [89]:
sys.getsizeof(words_emb_dict) / 1024 / 1024

10.000091552734375

In [95]:
# joblib
import joblib
joblib.dump(words_emb_dict,f'data/results/words_emb_dict_j')

['data/results/words_emb_dict_j']

In [94]:
for i,v in out.items():
    print(v['times'][0])

741.5214424000001
1851.9714044000002
3172.2670228999996
4569.112715900001
4641.669221300001
1953.6475882999985


In [108]:
for i,v in out.items():
    print(v['n_grams_list'][0])

[(2, 3)]
[(2, 3), (3, 4)]
[(2, 3), (3, 4)]
[(2, 3), (3, 4)]
[(2, 3), (3, 4)]
[(2, 3)]


In [111]:
[i for i in out.keys()]

['ess_1_keywords',
 'ess_2_keywords',
 'ess_3_keywords',
 'ess_4_keywords',
 'ess_5_keywords',
 'ess_6_keywords']

In [118]:
for essay in range(0,6):
# for i,v in out.items():
    save_obj(out[f'ess_{essay+1}_keywords'],f'data/results/keywords_res_essay_{essay+1}')
    # print(out[f'ess_{essay+1}_keywords'].keys())
    # print(f'data/results/keywords_res_essay_{i+1}')

In [104]:
for essay in range(6,10):
    # get top_n, diversity, ngram_range
    print("essayset",essay+1)
    
    model_candidates = list(map(lambda ans: key_words.candidates_tokens(ans,n_gram_range=ngram_range_l[essay+1]),ess_dict[f'ess_{essay+1}_model_answers']))
    model_candidate_emb = list(map(lambda cand:BERT.model.encode(cand),model_candidates))
    keywords = list(map(lambda x: maximal_marginal_relevance(
        x[0].reshape(1, -1),x[1],x[2],top_n=top_n_l[essay+1],diversity=diversity_l[essay+1]),
        zip(emb_dict[f'ess_{essay+1}_model_answers_emb'],
        model_candidate_emb,model_candidates)))
    students_n_grams = list(map(lambda keyword: sorted(key_words.get_n_grams(keyword)),keywords))

    docs = df.query(f'EssaySet == {essay+1}')["EssayText"].values.tolist()

    dd = para_keywords_pipeline_output_dict(keywords,docs,students_n_grams,batch=10)
    save_obj(dd,f'data/results/keywords_res_essay_{essay+1}')

    del dd

essayset 7
essayset 8
essayset 9
essayset 10


In [128]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [139]:
def predict(doc1,doc2):
    return float(cos_sim(model.encode(doc1),model.encode(doc2)))

In [149]:
docs = df.query(f'EssaySet == {1+1}')["EssayText"].values.tolist()

In [154]:
t1 = time.perf_counter()
# 749mb
x = model.encode(docs) # 30mb memory
print(time.perf_counter() - t1)

55.90439499999775


In [156]:
t1 = time.perf_counter()
# 750mb
x = BERT.model.encode(docs) # 100mb memory
time.perf_counter() - t1

162.5099635000006

In [148]:
print(predict('I am a student','I am hazem')) # 0.48043105006217957
print(predict('lava is the molten rock expelled by a volcano during an eruption',
    "الحمم البركانية هي الصخور المنصهرة التي طردها بركان أثناء ثوران بركان")) # 0.8797786831855774
print(predict('increases','decreases')) #0.3992762267589569
print(predict('تقل','decrease')) # 0.5011984705924988
print(predict('تنقص','decrease')) # 0.6617798805236816
print(predict('تخفيض','decrease')) # 0.8037792444229126


0.48043105006217957
0.8797786831855774
0.3992762267589569
0.5011984705924988
0.6617798805236816
0.8037792444229126


In [133]:
cos_sim(model.encode(["lava is the molten rock expelled by a volcano during an eruption"]),
model.encode(["الحمم البركانية هي الصخور المنصهرة التي طردها بركان أثناء ثوران بركان"]))


tensor([[0.8798]])

In [127]:
cos_sim(BERT.model.encode(["increases"]),BERT.model.encode(["decreases"]))

tensor([[0.7766]])

In [10]:
# Kriging-style spatial regression / inverse distance interpolation
import numpy as np
import random
random.seed(100)
# --- Highlights of this "fuzzy regression" code:
# Model-free; produces big output file to compute prediction intervals; Bivariate case,
# featuring nearest-neighbor approach (the weights); Exact predictions for training
# set, yet robust (no overfitting); Increasing M is "lazy way" to boost performance,
# but it slows speed
# Math-free (no matrix algrebra, square root or calculus); Statistics-free (no
# statistical science involved at all); Requires no technical knowledge beyond high
# school, but far from trivial!
# Acts as low-pass, amplitude reduction, or signal compression filter; Also acts as
# noise filtering, signal enhancement. Amplitude restoration step not included, but
# easy to do.
# By Vincent Granville, www.MLTechniques.com
# --- Hyperparameters
# n (number of obs, called points) set after reading input file [n=1000 here]
P=0.8 # proportion of data allocated to training the remaining is for validation
M=5000 # max number of splines used per point; M=5000 offers modest gain over M=800
r=2 # number of points defining a spline; also works with r=1 or larger r
smoother=1.5 # smoothing param used in weighted predictions; try 0.5 for more smoothing(0 = max smoothing)
thresh1=25.0 # max distance allowed to nearby spline; increase to eliminate points with no predictions;
# decrease to narrow (improve) confidence intervals
thresh2=1.5 # max outlier level allowed for predicted values ; if < 1, predicted can’t be more 
# extreme than observed; if too low, may increase number of points with no prediction; if too 
# large, may produce a few strong outlier predictions;
thresh3=0.001 # control numerical stability (keep 0.001)
# --- Output var (defined later)
# missing : number of points not assigned a prediction
#
# count : actual number of splines used for a specific point
# error : code telling why a point is not assigned a prediction
# weight : weight assigned to a spline, for a given point
# zpred : predicted value for a point zz = (xx, yy)
# zpredw : weighted predicted value
# Input var (defined later)
#
# xx, yy, zz: coordinates of a point

In [13]:
# --- Reading input file
x=[]
y=[]
z=[]
file=open('fuzzy2b.txt',"r")
lines=file.readlines()
for aux in lines:
    x.append(aux.split('\t')[0])
    y.append(aux.split('\t')[1])
    z.append(aux.split('\t')[2])
file.close()
x = list(map(float, x))
y = list(map(float, y))
z = list(map(float, z))
zmin=np.min(z)
zmax=np.max(z)
zavg=np.mean(z)
zdev=max(abs(zmin-zavg),abs(zmax-zavg))
n=len(x)

In [12]:
# --- Core function: spline-based interpolator
def F(xx,yy,r):
    zz=0
    distmin=1
    error=0
    idx=[]
    A=[]
    B=[]
    for i in range(0,r):
        idx.insert(i,int(n*P*random.random()))
    prod=1.0;
    for i in range(0,r):
        for j in range(i+1,r):
            prod*=(x[idx[i]]-x[idx[j]])*(y[idx[i]]-y[idx[j]])
    if abs(prod)>thresh3:
        for i in range(0,r):
            A.insert(i,1.0)
            B.insert(i,1.0)
            for j in range(0,r):
                if j != i:
                    A[i]*=(xx-x[idx[j]])/(x[idx[i]]-x[idx[j]])
                    B[i]*=(yy-y[idx[j]])/(y[idx[i]]-y[idx[j]])
            zz+=z[idx[i]]*(A[i]+B[i])/2
            distmin*=max(abs(xx-x[idx[i]]),abs(yy-y[idx[i]]))
        distmin=pow(distmin,1/r)
    else:
        error=1;
    return [zz,distmin,error]

In [16]:
import math

#----------------------------------------------------
# Main step: predictions for points in validation set
# For training set points, change range(int(P*n),n) to range(0,int(P*n))

file_small=open("fuzzy_small.txt","w")
file_big=open("fuzzy_big.txt","w")

for j in range(int(P*n),n): # loop over all validation points 

  xx=x[j]
  yy=y[j]
  zobs=z[j]
  count=0
  missing=0
  sweight=0.0
  zpredw=0.0
  zpred=0.0

  for k in range(0,M): # inner loop over all splines

    list=F(xx,yy,r)
    zz=list[0]
    distmin=list[1]
    error=list[2]
    weight=math.exp(-smoother*distmin)  
    zzdevratio=abs(zz-zavg)/zdev

    if distmin<thresh1 and zzdevratio<thresh2 and error==0: 
      count+=1
      sweight+=weight
      zpredw+=zz*weight
      zpred+=zz
      row=[j,xx,yy,zobs,zz,distmin,weight,zzdevratio]
      for field in row:
        file_big.write(str(field)+"\t")
      file_big.write("\n")

  if count>0:
    zpredw=zpredw/sweight
    zpred=zpred/count
  else:
    missing+=1
    zpredw=""
    zpred=""

  row=[j,count,xx,yy,zobs,zpred,zpredw] 
  for field in row:
    file_small.write(str(field)+"\t")
  file_small.write("\n")

file_big.close()
file_small.close()
print(missing,"ignored points\n")

0 ignored points

